# Test PySyft x F2D

In [1]:
import syft as sy

In [2]:
sy.load("sklearn")
sy.load("pandas")
sy.load("numpy")

In [3]:
sy.load("fhir2dataset")

In [4]:
remote = sy.VirtualMachine().get_root_client()

[2021-04-05T17:19:51.641470+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 0.8.2
[2021-04-05T17:19:51.642341+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.autocontrast not supported in 0.8.2
[2021-04-05T17:19:51.642895+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.equalize not supported in 0.8.2
[2021-04-05T17:19:51.643667+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.invert not supported in 0.8.2
[2021-04-05T17:19:51.644166+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.posterize not supported in 0.8.2
[2021-04-05T17:19:51.644934+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.solarize not supported in 0.8.2


In [5]:
remote.fhir2dataset

Module:
	.Query -> <syft.ast.klass.Class object at 0x13e95c8e0>

In [6]:
from fhir2dataset import Query


query = Query(fhir_api_url="http://hapi.fhir.org/baseR4/")

## Test serialization

In [8]:
d = sy.serialize(obj=query)

In [9]:
type(d)

proto.lib.fhir2dataset.query_pb2.FHIRQuery

In [10]:
sy.deserialize(blob=sy.serialize(obj=query))

# Create remotely a Query

In [11]:
remote_query = remote.fhir2dataset.Query(fhir_api_url="http://hapi.fhir.org/baseR4/")

In [13]:
sql_query = """
SELECT
Patient.birthDate,
Condition.code.coding.display
FROM Condition
INNER JOIN Patient 
ON Condition.subject = Patient._id
WHERE Patient.birthdate=ge2000-01-01
AND Patient.birthdate=le2000-04-09
"""

In [16]:
response = remote_query.from_sql(sql_query)

100%|█████████▉| 100.00/100.00


In [17]:
response

In [18]:
response.get()

,Condition:from_id,Condition:Condition.code.coding.display,Patient:from_id,Patient:Patient.birthDate
0,Condition/1656578,rheumatoid arthritis,Patient/1656571,2000-01-01
1,Condition/1656577,neuropathic pain,Patient/1656571,2000-01-01
2,Condition/1656579,rheumatoid arthritis,Patient/1656571,2000-01-01
3,Condition/1656574,diabetes,Patient/1656571,2000-01-01
4,Condition/1656573,asthma,Patient/1656571,2000-01-01
...,...,...,...,...
412,Condition/1499252,rheumatoid arthritis,Patient/1499244,2000-02-01
413,Condition/1499251,rheumatoid arthritis,Patient/1499244,2000-02-01
414,Condition/1499254,high cholesterol,Patient/1499244,2000-02-01
415,Condition/1499253,GERD,Patient/1499244,2000-02-01


# Send a query

In [24]:
alice_vm = sy.VirtualMachine(name="alice")

# Get clients from each VM
alice = alice_vm.get_root_client()

query = Query(fhir_api_url="http://hapi.fhir.org/baseR4/")
query.tag('#f2d')

query_ptr = query.send(alice)

[2021-04-05T17:22:33.493440+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 0.8.2
[2021-04-05T17:22:33.494215+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.autocontrast not supported in 0.8.2
[2021-04-05T17:22:33.494775+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.equalize not supported in 0.8.2
[2021-04-05T17:22:33.495671+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.invert not supported in 0.8.2
[2021-04-05T17:22:33.496377+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.posterize not supported in 0.8.2
[2021-04-05T17:22:33.497275+0200][CRITICAL][logger]][44853] Skipping torchvision.torchvision.transforms.functional.solarize not supported in 0.8.2


In [18]:
query_ptr

In [ ]:
alice.store[0].tags

In [19]:
query_back = query_ptr.get()

In [ ]:
query_back

In [20]:
query_back.tags

['#f2d']